In [34]:
import pandas as pd

In [35]:
df = pd.read_csv('/content/train.csv',on_bad_lines="skip", engine='python')

In [36]:
df.head(10)

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1
5,5,Jackie Mason: Hollywood Would Love Trump if He...,Daniel Nussbaum,"In these trying times, Jackie Mason is the Voi...",0
6,6,Life: Life Of Luxury: Elton John’s 6 Favorite ...,NaN,Ever wonder how Britain’s most iconic pop pian...,1
7,7,Benoît Hamon Wins French Socialist Party’s Pre...,Alissa J. Rubin,"PARIS — France chose an idealistic, traditi...",0
8,8,Excerpts From a Draft Script for Donald Trump’...,NaN,Donald J. Trump is scheduled to make a highly ...,0
9,9,"A Back-Channel Plan for Ukraine and Russia, Co...",Megan Twohey and Scott Shane,A week before Michael T. Flynn resigned as nat...,0


In [37]:
df.isnull().sum()

id           0
title      565
author    1975
text        59
label       22
dtype: int64

In [38]:
df.shape

(20822, 5)

In [39]:
df=df.dropna()

In [40]:
df.shape

(18285, 5)

In [41]:
# Check for rows where the dependent variable is not numeric
invalid_rows = df[~df['label'].apply(lambda x: str(x).isdigit())]
print("Invalid rows:", invalid_rows)

Invalid rows:                                                       id  \
19781  Мы с Всемирным банком проводим каждые пять лет...   
19782            Хотя финансовые ограничения тоже вредны   

                                                   title  \
19781                               и в Восточной Европе   
19782   они вызывают определённую нервозность у инвес...   

                                                  author  \
19781   теперь в Турции и на Ближнем Востоке. И после...   
19782   о которых Вы сказали. И мне не нравится. А ко...   

                                      text  \
19781   и в том числе там был вопрос о том   
19782     чтобы этого не было. Надо только   

                                                  label  
19781             как люди воспринимают своё положение.  
19782   чтобы это была дорога с двусторонним движением.  


In [42]:
# Filter out non-numeric rows
df = df[df['label'].apply(lambda x: str(x).isdigit())]

# Convert 'dependent_var' to integer type
df['label'] = df['label'].astype(int)

In [43]:
## Indepedent Feature
X=df.drop('label', axis=1)

In [44]:
## Dependent featur
y=df['label']

In [45]:
## check whether dataset is balanced or not. checking the output column.
y.value_counts()

label
0    10361
1     7922
Name: count, dtype: int64

In [46]:
import tensorflow as tf

In [47]:
tf.__version__

'2.15.0'

In [48]:
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Bidirectional

In [49]:
## vocabular size
voc_size = 5000

In [50]:
messages = X.copy()

In [51]:
messages.head(10)

,id,title,author,text
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ..."
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...
5,5,Jackie Mason: Hollywood Would Love Trump if He...,Daniel Nussbaum,"In these trying times, Jackie Mason is the Voi..."
7,7,Benoît Hamon Wins French Socialist Party’s Pre...,Alissa J. Rubin,"PARIS — France chose an idealistic, traditi..."
9,9,"A Back-Channel Plan for Ukraine and Russia, Co...",Megan Twohey and Scott Shane,A week before Michael T. Flynn resigned as nat...
10,10,Obama’s Organizing for Action Partners with So...,Aaron Klein,"Organizing for Action, the activist group that..."
11,11,"BBC Comedy Sketch ""Real Housewives of ISIS"" Ca...",Chris Tomlinson,The BBC produced spoof on the “Real Housewives...


In [52]:
import nltk
import re
from nltk.corpus import stopwords

In [53]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [54]:
messages.reset_index(inplace=True)

In [55]:
### Dataset Preprocessing
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()
corpus = []
for i in range(0, len(messages)):
    review = re.sub('[^a-zA-Z]', ' ', messages['title'][i])
    review = review.lower()
    review = review.split()

    review = [ps.stem(word) for word in review if not word in stopwords.words('english')]
    review = ' '.join(review)
    corpus.append(review)

In [56]:
corpus

['hous dem aid even see comey letter jason chaffetz tweet',
 'flynn hillari clinton big woman campu breitbart',
 'truth might get fire',
 'civilian kill singl us airstrik identifi',
 'iranian woman jail fiction unpublish stori woman stone death adulteri',
 'jacki mason hollywood would love trump bomb north korea lack tran bathroom exclus video breitbart',
 'beno hamon win french socialist parti presidenti nomin new york time',
 'back channel plan ukrain russia courtesi trump associ new york time',
 'obama organ action partner soro link indivis disrupt trump agenda',
 'bbc comedi sketch real housew isi caus outrag',
 'russian research discov secret nazi militari base treasur hunter arctic photo',
 'us offici see link trump russia',
 'ye paid govern troll social media blog forum websit',
 'major leagu soccer argentin find home success new york time',
 'well fargo chief abruptli step new york time',
 'anonym donor pay million releas everyon arrest dakota access pipelin',
 'fbi close hilla

In [57]:
## One hot representation
onehot_repr=[one_hot(words,voc_size) for words in corpus]
onehot_repr

[[4426, 4414, 498, 3836, 2432, 3517, 3163, 1321, 3345, 1106],
 [315, 4909, 3250, 4104, 3609, 3558, 4041],
 [3404, 3937, 4162, 4867],
 [4134, 4006, 293, 1650, 4320, 4343],
 [1696, 3609, 1144, 1240, 55, 392, 3609, 4196, 965, 1015],
 [1071,
  2168,
  4594,
  4522,
  2551,
  4095,
  1715,
  3761,
  1098,
  2489,
  4553,
  3199,
  1334,
  1437,
  4041],
 [4354, 2262, 3906, 35, 4941, 408, 1400, 4773, 1168, 3346, 2095],
 [2146, 4935, 2500, 2911, 3846, 254, 4095, 708, 1168, 3346, 2095],
 [342, 3778, 3088, 3080, 4060, 786, 3035, 50, 4095, 4847],
 [1247, 2069, 790, 793, 2906, 962, 4889, 223],
 [819, 1340, 4239, 3402, 1733, 2509, 520, 700, 4703, 2023, 792],
 [1650, 1444, 2432, 786, 4095, 3846],
 [473, 2187, 638, 2507, 2947, 585, 4949, 3811, 3237],
 [431, 4138, 3728, 2193, 2474, 2313, 4113, 1168, 3346, 2095],
 [4187, 4168, 2977, 3260, 1663, 1168, 3346, 2095],
 [2052, 2272, 3192, 3387, 452, 1239, 2318, 3276, 3664, 3864],
 [2173, 1479, 4909],
 [446, 2105, 939, 4357, 4095, 1128, 2217, 4041],
 [3797, 

In [58]:
## Padding and embedding layer
sent_length=20
embedded_docs=pad_sequences(onehot_repr, padding='post', maxlen=sent_length)

embedded_docs

array([[4426, 4414,  498, ...,    0,    0,    0],
       [ 315, 4909, 3250, ...,    0,    0,    0],
       [3404, 3937, 4162, ...,    0,    0,    0],
       ...,
       [3611, 3017, 4848, ...,    0,    0,    0],
       [1190, 3846, 4807, ...,    0,    0,    0],
       [ 775, 2415,  748, ...,    0,    0,    0]], dtype=int32)

In [59]:
## Creating model
embedding_vector_features=40 #feature representation
model=Sequential()
model.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
model.add(Bidirectional(LSTM(100)))
#100 indicates 100 neurons in forward, 100 neurons in backward, total=200
model.add(Dense(1,activation='sigmoid')) #output layer
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
print(model.summary())

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 20, 40)            200000    
                                                                 
 bidirectional_1 (Bidirecti  (None, 200)               112800    
 onal)                                                           
                                                                 
 dense_1 (Dense)             (None, 1)                 201       
                                                                 
Total params: 313001 (1.19 MB)
Trainable params: 313001 (1.19 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
None


In [60]:
## converting into array to supply it for training
import numpy as np
X_final = np.array(embedded_docs)
y_final = np.array(y)

In [61]:
## train test split

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_final, y_final, test_size=0.33, random_state=42)

In [62]:
## Model Training
model.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=10,batch_size=32)

Epoch 1/10
383/383 [==============================] - 25s 54ms/step - loss: 0.2630 - accuracy: 0.8842 - val_loss: 0.1843 - val_accuracy: 0.9216
Epoch 2/10
383/383 [==============================] - 17s 46ms/step - loss: 0.1191 - accuracy: 0.9553 - val_loss: 0.2016 - val_accuracy: 0.9209
Epoch 3/10
383/383 [==============================] - 20s 51ms/step - loss: 0.0754 - accuracy: 0.9722 - val_loss: 0.2462 - val_accuracy: 0.9130
Epoch 4/10
383/383 [==============================] - 18s 46ms/step - loss: 0.0454 - accuracy: 0.9855 - val_loss: 0.2773 - val_accuracy: 0.9120
Epoch 5/10
383/383 [==============================] - 18s 47ms/step - loss: 0.0288 - accuracy: 0.9913 - val_loss: 0.3260 - val_accuracy: 0.9112
Epoch 6/10
383/383 [==============================] - 18s 46ms/step - loss: 0.0200 - accuracy: 0.9934 - val_loss: 0.3839 - val_accuracy: 0.9085
Epoch 7/10
383/383 [==============================] - 18s 47ms/step - loss: 0.0160 - accuracy: 0.9949 - val_loss: 0.4499 - val_accuracy:

In [63]:
y_pred=model.predict(X_test)

189/189 [==============================] - 4s 16ms/step


In [64]:
import numpy as np

y_pred=np.where(y_pred>=0.5,1,0)

In [65]:
from sklearn.metrics import confusion_matrix,accuracy_score
confusion_matrix(y_test,y_pred)

array([[3090,  325],
       [ 225, 2394]])

In [66]:
print(accuracy_score(y_test,y_pred))

0.9088498508452105
